In [8]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [26]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [27]:
"""BFFA"""
def exchange_binary(binary,score):#,alpha,beta,gamma,r):

    #binary in list
    al_binary=binary
    #movement=move(b,alpha,beta,gamma,r)
    movement=math.tanh(score)
    ##al_binary=[case7(b) if random.uniform(0,1) < movement else case8(b) for b in binary]
    if random.uniform(0,1) < movement:
        for i,b in enumerate(binary):
            al_binary[i]=case7(b)
    else:
        for i,b in enumerate(binary):
            al_binary[i]=case8(b)
    return al_binary

def case7(one_bin):
    return 1 if random.uniform(-0.1,0.9)<math.tanh(one_bin) else 0
def case8(one_bin):
    if random.uniform(-0.1,0.9)<math.tanh(int(one_bin)):
        if one_bin==1:
            return 0
        else:return 1
    else:return one_bin
def case9(one_bin,best):
    if random.uniform(0,1)<math.tanh(int(one_bin)):
        return best
    else:return 0

def BFFA(Eval_Func,n=20,m_i=25,minf=0,dim=None,prog=False,gamma=1.0,beta=0.20,alpha=0.25,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)
    #flag=dr
    global_best=float("-inf") if minf == 0 else float("inf")
    pb=float("-inf") if minf == 0 else float("inf")

    global_position=tuple([0]*dim)
    gen=tuple([0]*dim)
    #gamma=1.0
    #beta=0.20
    #alpha=0.25
    gens_dict = {tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    #gens_dict[global_position]=0.001
    gens=random_search(n,dim)
    #vs = [[random.choice([0,1]) for i in range(length)] for i in range(N)]
    if mp!=None:
        with Pool(mp) as p:
            alter_fit = p.map(estimate,[gens[i] for i in alter_gens])
        for i,j in zip(alter_fit,gens):
            gens_dict[tuple(j)]=i
            fit[i]=i
        global_best=max(fit)
        global_position=gens[fit.index(max(fit))]

    else:
        for gen in gens:
            if tuple(gen) in gens_dict:
                score = gens_dict[tuple(gen)]
            else:
                score=estimate(gen)
                gens_dict[tuple(gen)]=score
            if score > global_best:
                global_best=score
                global_position=dc(gen)
    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)
    for it in miter:
        for i,x in enumerate(gens):
            if mp != None:
                for j,y in enumerate(gens):
                    if gens_dict[tuple(y)] < gens_dict[tuple(x)]:
                        gens[j]=exchange_binary(y,gens_dict[tuple(y)])
                    gen = gens[j]

                fit=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in gens]
                alter_gens=[i for i,g in enumerate(gens) if tuple(g) not in gens_dict]
                #print(len(alter_gens))
                with Pool(mp) as p:
                    alter_fit = p.map(estimate,[gens[i] for i in alter_gens])
                z=0
                for k in range(len(fit)):
                    if k in alter_gens:
                        fit[k]=alter_fit[z]
                        gens_dict[tuple(gens[k])]=alter_fit[z]
                    else:pass
                    if fit[k] > global_best if minf==0 else fit[k] < global_best:
                        global_best=dc(fit[k])
                        global_position=dc(gen)

            else:
                for j,y in enumerate(gens):
                    if gens_dict[tuple(y)] < gens_dict[tuple(x)]:
                        gens[j]=exchange_binary(y,gens_dict[tuple(y)])
                    gen = gens[j]
                    if tuple(gen) in gens_dict:
                        score = gens_dict[tuple(gen)]
                    else:
                        score=estimate(gens[j])
                        gens_dict[tuple(gen)]=score
                    if score > global_best if minf==0 else score < global_best:
                        global_best=dc(score)
                        global_position=dc(gen)
    print(len(gens_dict))
    return global_best,global_position,global_position.count(1)

In [28]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("CSS10_Spec128.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

7000 129


In [29]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [30]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [31]:
%%time
s,g,l=BFFA(Eval_Func=Evaluate, n=1, m_i=1)

2
CPU times: user 343 ms, sys: 12.3 ms, total: 355 ms
Wall time: 431 ms


In [32]:
l

59

In [33]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("CSS10_Spec128.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,label
0,0.000725,0.000390,0.001063,0.016048,0.045006,0.088202,0.050937,0.048269,0.081694,0.061398,...,0.040290,0.035810,0.036298,0.038804,0.033180,0.036938,0.021571,0.009373,6.623209e-03,Chinese
1,0.005059,0.002901,0.005581,0.026885,0.217670,0.280215,0.095449,0.109636,0.252080,0.268016,...,0.205824,0.407305,0.610883,0.180494,0.089081,0.081824,0.057899,0.032875,1.317016e-02,Chinese
2,0.050511,0.033339,0.020180,0.046618,0.193343,0.242709,0.097945,0.239527,0.509217,0.336884,...,0.026522,0.028574,0.018005,0.016250,0.012089,0.009391,0.008720,0.006136,4.109685e-03,Chinese
3,0.001279,0.000901,0.003375,0.041863,0.200739,0.168802,0.106558,0.527622,0.437945,0.274857,...,0.023716,0.014288,0.017838,0.018208,0.014407,0.010028,0.007528,0.005653,4.462213e-03,Chinese
4,0.001710,0.001331,0.002648,0.027608,0.163129,0.289433,0.170620,0.243592,0.555679,0.161425,...,0.209389,0.256350,0.212744,0.145164,0.106921,0.067407,0.049716,0.025963,1.462485e-02,Chinese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.057926,2.879856,20.679771,20.195250,0.379650,1.958890,2.496896,5.655553,6.943397,5.476128,...,0.008678,0.006215,0.011819,0.014670,0.011008,0.007485,0.007760,0.001036,2.610000e-07,Spanish
6996,0.078999,4.854642,16.547379,10.877720,0.576475,3.127383,4.036949,5.273048,4.024424,6.352676,...,0.007153,0.005911,0.006234,0.005297,0.008319,0.005898,0.005996,0.001108,2.550000e-07,Spanish
6997,0.236640,3.645541,11.302727,16.783052,0.851596,2.084428,4.140211,3.063443,2.992726,4.451303,...,0.017653,0.019413,0.007211,0.006127,0.004149,0.002933,0.002529,0.000331,7.720000e-08,Spanish
6998,0.118839,2.011996,15.797884,18.321749,0.504213,1.365581,4.591431,4.060121,7.282707,6.573135,...,0.004784,0.004022,0.008535,0.008248,0.010245,0.007024,0.006153,0.000721,1.490000e-07,Spanish


In [34]:
df_train1.to_csv('BFFA128.csv')